In [2]:
# Connecting to MS SQL Server from Python with PyODBC
# pip install pyodbc

In [ ]:
# Connecting to MS SQL Server

import pyodbc

def create_server_connection(driver, server, database, user, password):
    conn = None
    try:
                conn = pyodbc.connect(DRIVER = driver, SERVER = server, DATABASE = database, UID = user, \
                                      PWD = password)
                print("MS SQL Server Database connection successful")
    except Error as err:
        print(f"Error: '{err}'")
    return conn


driver = '{ODBC Driver 17 for SQL Server}'
server = 'localhost' 
database = 'master' 
user = 'sa' 
password = 'xxx' # kendi password'umuz
conn = create_server_connection(driver, server, database, user, password)

# MS SQL Server Database connection successful

In [ ]:
# Creating a New Database

def create_database(conn, create_database_query):
    csr = conn.cursor()
    try:
        csr.execute(create_database_query)
        print("Database created successfully")
    except Error as err:
        print(f"Error: '{err}'")
    

conn.autocommit = True
create_database_query = 'CREATE DATABASE TestDB'
create_database(conn, create_database_query)

# Database created successfully

In [ ]:
# Creating a Query Execution Function

def execute_query(conn, query):
    csr = conn.cursor()
    try:
        csr.execute(query)
        conn.commit()
        print("Query successful")
    
    except Error as err:
        print(f"Error: '{err}'")


query ='USE TestDB'
execute_query(conn, query)

# Query successful

query = 'CREATE TABLE TestA ( \
             ID int IDENTITY(1,1) NOT NULL, \
             FirstName varchar(255) NOT NULL, \
             LastName varchar(255) NOT NULL, \
             PRIMARY KEY(ID))'
execute_query(conn, query)

# Query successful

In [ ]:
# Inserting Data

csr = conn.cursor()

csr.execute("INSERT TestA (FirstName, LastName) VALUES ('Aaaa', 'Bbbb')")

conn.commit() # yukarida autocommit yaptigimiz icin buna aslinda gerek yok

In [ ]:
# or, parameterized:
csr.execute("INSERT TestA (FirstName, LastName) VALUES (?, ?)", 'Cccc', 'Dddd')

conn.commit()

In [ ]:
# Updating and Deleting

csr.execute("delete from TestA where FirstName = ?", 'Aaaa')

conn.commit()

print(csr.rowcount, 'row(s) deleted')

In [ ]:
deleted = csr.execute("delete from TestA where FirstName = 'Cccc'").rowcount

conn.commit()

print(deleted, 'row(s) deleted')

### Reading Data

In [ ]:
csr = conn.cursor()
csr.execute('select FirstName, LastName from TestA')
row = csr.fetchone()
if row:
    print(row)

In [ ]:
# Row objects are similar to tuples, but they also allow access to columns by name:

csr.execute('select FirstName, LastName from TestA')
row = csr.fetchone()
print('LastName:', row[1]) # access by column index (zero-based)
print('LastName:', row.LastName) # access by name

In [ ]:
# The fetchone() function returns None when all rows have been retrieved.

csr.execute('select FirstName, LastName from TestA')
while True:
    row = csr.fetchone()
    if not row:
        break
    print('FirstName:', row.FirstName)

```
The fetchall() function returns all remaining rows in a list. Bear in mind those rows will all be stored in memory so if there are a lot of rows, you may run out of memory.

If there are no rows, an empty list is returned.
```

In [ ]:
csr.execute('select FirstName, LastName from TestA')
rows = csr.fetchall()

In [ ]:
for row in rows:
    print(row.FirstName, row.LastName)

In [ ]:
# If you are going to process the rows one at a time, you can use the cursor itself as an iterator:

csr.execute('select FirstName, LastName from TestA')
for row in csr:
    print(row.FirstName, row.LastName)

#or just:

for row in csr.execute('select FirstName, LastName from TestA'):
    print(row.FirstName, row.LastName)

```
ODBC supports parameters using a question mark as a place holder in the SQL.
You provide the values for the question marks by passing them after the SQL:
```

In [ ]:
csr.execute("""select FirstName, LastName from TestA
               where FirstName = ?""", 'Aaaa')
row = csr.fetchone()
if row:
    print(row)

In [ ]:
# The Python DB API specifies that parameters should be passed as a sequence, so this is also supported by pyodbc:

csr.execute(""" select FirstName, LastName from TestA
		where FirstName = ? and LastName = ? """, ['Aaaa', 'Bbbb'])
row = csr.fetchone()
if row:
    print(row)

In [ ]:
# To simplify the process, a function def read_query function can be created.

def read_query(conn, query):
    csr = conn.cursor()
    results = None
    try:
        csr.execute(query)
        results = csr.fetchall()
        # Close and delete cursor
        csr.close()
        del csr
        return results    
    except Error as err:
        print(f"Error: '{err}'")

query = 'select FirstName, LastName from TestA'
execute_query(conn, query) -- kodu yukarida var
results = read_query(conn, query)
print(results)
conn.close()

import pandas as pd
from_db = []

for result in results:
    result = list(result)
    from_db.append(result)

columns = ['FirstName', 'LastName']
df = pd.DataFrame(from_db, columns=columns)
display(df)

### Tips and Tricks

In [ ]:
# Since single quotes are valid in SQL, use double quotes to surround your SQL:

deleted = csr.execute("delete from TestA where FirstName = 'Aaaa'").rowcount

```
It's also worthwhile considering using 'raw' strings for your SQL to avoid any inadvertent escaping (unless you really do want to specify control characters):
```

In [ ]:
csr.execute("delete from TestA where FirstName like '%bad\name%'") # Python will convert \n to 'new line'!

csr.execute(r"delete from TestA where name like '%bad\name%'") # no escaping

```
Some databases (e.g. SQL Server) do not generate column names for calculated fields, e.g. `COUNT(*)`.

In that case you can either access the column by its index, or use an alias on the column (i.e. use the "as" keyword).
```

In [ ]:
row = csr.execute("select count(*) as user_count from TestA").fetchone()

print('%s users' % row.user_count)

In [ ]:
# Formatting Long SQL Statements

import textwrap

sql = textwrap.dedent("""select FirstName, LastName from TestA
                         where FirstName = 'Aaaa' and LastName = ?
                      """)
rows = csr.execute(sql, 'Bbbb').fetchall()

In [ ]:
# fetchval

maxid = csr.execute("select max(ID) from TestA").fetchval()

```
Most databases support `COALESCE` or `ISNULL` which can be used to convert `NULL` to a hardcoded value, but note that this will not cause a row to be returned if the SQL returns no rows.

That is, `COALESCE` is great with aggregate functions like max or count, but fetchval is better when attempting to retrieve the value from a particular row:
```

In [ ]:
csr.execute("select coalesce(max(ID), 0) from TestA where FirstName = 'Eeee'").fetchone()[0]

csr.execute("select coalesce(count(*), 0) from TestA where FirstName = 'Eeee'").fetchone()[0]

In [ ]:
# However, fetchval is a better choice if the statement can return an empty set:
# Careful !!!

csr.execute("""select FirstName from TestA where FirstName = 'Eeee'
            """).fetchone()[0]

# Preferred
csr.execute("""select FirstName from TestA where FirstName = 'Eeee'
            """).fetchval()

```
The first example will raise an exception if there are no rows.

The fetchone() call returns None. Python then attempts to apply [0] to the result (None[0]) which is not valid.

The fetchval method was created just for this situation - it will detect the fact that there are no rows and will return None.
```